In [ ]:
# default_exp data.mixed

# Mixed data

> DataLoader than can take data from multiple dataloaders with different types of data

In [ ]:
#export
from tsai.imports import *

In [ ]:
# export
# This implementation of a mixed dataloader is based on a great implementation created by Zach Mueller in this fastai thread:
# https://forums.fast.ai/t/combining-tabular-images-in-fastai2-and-should-work-with-almost-any-other-type/73197

from packaging import version
from fastai.data.load import _FakeLoader
from torch.utils.data.dataloader import _MultiProcessingDataLoaderIter, _SingleProcessDataLoaderIter, _DatasetKind
_loaders = (_MultiProcessingDataLoaderIter, _SingleProcessDataLoaderIter)


class MixedDataLoader():
    def __init__(self, *loaders, path='.', shuffle=False, device=None, bs=None):
        "Accepts any number of `DataLoader` and a device"
        self.path = path
        device = ifnone(device, default_device())
        self.device = device
        self.c = None
        self.bs = ifnone(bs, min([dl.bs for dl in loaders]))
        for i, dl in enumerate(loaders):  # ensure all dls have the same bs
            if hasattr(dl, 'vars'):
                self.vars = dl.vars
            if hasattr(dl, 'len'):
                self.len = dl.len
            if hasattr(dl, 'split_idxs'):
                self.split_idxs = dl.split_idxs
            dl.bs = self.bs
            dl.shuffle_fn = self.shuffle_fn
            if self.c is None and hasattr(dl, "c"):
                self.c = dl.c
            if i == 0:
                self.dataset = dl.dataset
            dl.to(device=device)
        self.shuffle = shuffle
        if not self.shuffle:
            self.rng = np.arange(len(self.dataset)).tolist()
        self.loaders = loaders
        self.count = 0
        self.fake_l = _FakeLoader(self, False, 0, 0, 0) if version.parse(
            fastai.__version__) >= version.parse("2.1") else _FakeLoader(self, False, 0, 0)
        if sum([len(dl.dataset) for dl in loaders]) > 0:
            self._get_idxs()  # Do not apply on an empty dataset

    def new(self, *args, **kwargs):
        loaders = [dl.new(*args, **kwargs) for dl in self.loaders]
        return type(self)(*loaders, path=self.path, device=self.device)

#     def __len__(self): return len(self.loaders[0])
    def __len__(self): return self.loaders[0].__len__()

    def _get_vals(self, x):
        "Checks for duplicates in batches"
        idxs, new_x = [], []
        for i, o in enumerate(x):
            x[i] = o.cpu().numpy().flatten()
        for idx, o in enumerate(x):
            if not self._arrayisin(o, new_x):
                idxs.append(idx)
                new_x.append(o)
        return idxs

    def _get_idxs(self):
        "Get `x` and `y` indices for batches of data"
        self.n_inps = [dl.n_inp for dl in self.loaders]
        self.x_idxs = self._split_idxs(self.n_inps)

        # Identify duplicate targets
        dl_dict = dict(zip(range(0, len(self.loaders)), self.n_inps))
        outs = L([])
        for key, n_inp in dl_dict.items():
            b = next(iter(self.loaders[key]))
            outs += L(b[n_inp:])
        self.y_idxs = self._get_vals(outs)

    def __iter__(self):
        z = zip(*[_loaders[i.fake_l.num_workers == 0](i.fake_l) for i in self.loaders])
        for b in z:
            inps = []
            outs = []
            if self.device is not None:
                b = to_device(b, self.device)
            for batch, dl in zip(b, self.loaders):
                if hasattr(dl, 'idxs'): self.idxs = dl.idxs
                if hasattr(dl, 'input_idxs'): self.input_idxs = dl.input_idxs
                batch = dl.after_batch(batch)
                inps += batch[:dl.n_inp]
                outs += batch[dl.n_inp:]
            inps = tuple([tuple(L(inps)[idx]) if isinstance(idx, list) else inps[idx]
                          for idx in self.x_idxs]) if len(self.x_idxs) > 1 else tuple(L(outs)[self.x_idxs][0])
            try: 
                outs = tuple(L(outs)[self.y_idxs]) if len(self.y_idxs) > 1 else L(outs)[self.y_idxs][0]
                yield inps, outs
            except IndexError: yield inps

    def one_batch(self):
        "Grab one batch of data"
        with self.fake_l.no_multiproc():
            res = first(self)
        if hasattr(self, 'it'):
            delattr(self, 'it')
        return res

    def shuffle_fn(self, idxs):
        "Generate the same idxs for all dls in each batch when shuffled"
        if self.count == 0:
            self.shuffled_idxs = np.random.permutation(idxs)
        # sort each batch
        for i in range(len(self.shuffled_idxs)//self.bs + 1):
            self.shuffled_idxs[i*self.bs:(i+1)*self.bs] = np.sort(self.shuffled_idxs[i*self.bs:(i+1)*self.bs])
        self.count += 1
        if self.count == len(self.loaders):
            self.count = 0
        return self.shuffled_idxs

    def show_batch(self):
        "Show a batch of data"
        for dl in self.loaders:
            dl.show_batch()

    def to(self, device): self.device = device

    def _arrayisin(self, arr, arr_list):
        "Checks if `arr` is in `arr_list`"
        for a in arr_list:
            if np.array_equal(arr, a):
                return True
        return False

    def _split_idxs(self, a):
        a_cum = np.array(a).cumsum().tolist()
        b = np.arange(sum(a)).tolist()
        start = 0
        b_ = []
        for i, idx in enumerate(range(len(a))):
            end = a_cum[i]
            b_.append(b[start:end] if end - start > 1 else b[start])
            start = end
        return b_
    

class MixedDataLoaders(DataLoaders):
    pass

In [ ]:
# export

def get_mixed_dls(*dls, device=None, shuffle_train=None, shuffle_valid=None, **kwargs):
    _mixed_train_dls = []
    _mixed_valid_dls = []
    for dl in dls:
        _mixed_train_dls.append(dl.train)
        _mixed_valid_dls.append(dl.valid)
        if shuffle_train is None: shuffle_train = dl.train.shuffle
        if shuffle_valid is None: shuffle_valid = dl.valid.shuffle
        if device is None: device = dl.train.device
    mixed_train_dl = MixedDataLoader(*_mixed_train_dls, shuffle=shuffle_train, **kwargs)
    mixed_valid_dl = MixedDataLoader(*_mixed_valid_dls, shuffle=shuffle_valid, **kwargs)
    mixed_dls = MixedDataLoaders(mixed_train_dl, mixed_valid_dl, device=device)
    return mixed_dls

In [ ]:
from tsai.data.tabular import *

path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
# df['salary'] = np.random.rand(len(df)) # uncomment to simulate a cont dependent variable
target = 'salary'
splits = RandomSplitter()(range_of(df))

cat_names = ['workclass', 'education', 'marital-status']
cont_names = ['age', 'fnlwgt']
dls1 = get_tabular_dls(df, cat_names=cat_names, cont_names=cont_names, y_names=target, splits=splits, bs=512)
dls1.show_batch()

cat_names = None #['occupation', 'relationship', 'race']
cont_names = ['education-num']
dls2 = get_tabular_dls(df, cat_names=cat_names, cont_names=cont_names, y_names=target, splits=splits, bs=128)
dls2.show_batch()

,workclass,education,marital-status,age,fnlwgt,salary
0,Private,HS-grad,Never-married,20.000000,333505.003512,<50k
1,Local-gov,Masters,Divorced,45.000000,93535.002309,<50k
2,Private,10th,Married-civ-spouse,57.999999,234481.000650,<50k
3,Private,Masters,Married-civ-spouse,49.000000,123680.997085,>=50k
4,Private,Bachelors,Married-civ-spouse,36.000000,68272.995723,>=50k
5,Private,Doctorate,Married-spouse-absent,31.000000,196025.000211,<50k
6,Private,Some-college,Divorced,30.000000,161815.001537,<50k
7,Self-emp-inc,HS-grad,Married-civ-spouse,45.000000,117605.001136,>=50k
8,Federal-gov,Some-college,Married-civ-spouse,53.999999,114673.998909,>=50k
9,Private,Some-college,Never-married,20.000000,215232.000557,<50k


,education-num_na,education-num,salary
0,False,9.0,<50k
1,False,6.0,<50k
2,False,9.0,<50k
3,False,13.0,<50k
4,False,13.0,>=50k
5,True,10.0,<50k
6,False,10.0,<50k
7,False,4.0,<50k
8,False,10.0,<50k
9,False,9.0,<50k


In [ ]:
dls = get_mixed_dls(dls1, dls2, bs=8)
first(dls.train)
first(dls.valid)
torch.save(dls,'export/mixed_dls.pth')
del dls
dls = torch.load('export/mixed_dls.pth')
dls.train.show_batch()

,workclass,education,marital-status,age,fnlwgt,salary
0,Private,Some-college,Never-married,23.000000,67958.004646,<50k
1,Private,Assoc-voc,Divorced,42.000000,168515.000677,<50k
2,Private,HS-grad,Never-married,19.000001,365639.998782,<50k
3,Private,Assoc-acdm,Never-married,25.000000,91709.002441,<50k
4,Private,5th-6th,Married-civ-spouse,33.000000,283602.001704,<50k
5,Private,7th-8th,Married-civ-spouse,83.999999,388384.003536,<50k
6,Private,Some-college,Never-married,46.000000,180694.999707,<50k
7,Private,Masters,Never-married,30.000000,55480.999910,<50k


,education-num_na,education-num,salary
0,True,10.0,<50k
1,False,11.0,<50k
2,False,9.0,<50k
3,False,12.0,<50k
4,False,3.0,<50k
5,False,4.0,<50k
6,False,10.0,<50k
7,False,14.0,<50k


In [ ]:
xb, yb = first(dls.train)
xb

((tensor([[ 5, 16,  5],
          [ 5,  9,  1],
          [ 5, 12,  5],
          [ 5,  8,  5],
          [ 5,  5,  3],
          [ 5,  6,  3],
          [ 5, 16,  5],
          [ 5, 13,  5]], device='cuda:0'),
  tensor([[-1.1389, -1.1568],
          [ 0.2512, -0.1987],
          [-1.4316,  1.6795],
          [-0.9926, -0.9305],
          [-0.4073,  0.8978],
          [ 3.3240,  1.8962],
          [ 0.5438, -0.0827],
          [-0.6268, -1.2757]], device='cuda:0')),
 (tensor([[2],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1]], device='cuda:0'),
  tensor([[-0.0268],
          [ 0.3635],
          [-0.4171],
          [ 0.7538],
          [-2.7587],
          [-2.3685],
          [-0.0268],
          [ 1.5343]], device='cuda:0')))

In [ ]:
xs, ys = first(dls.train)
xs[0][0].shape, xs[0][1].shape, xs[1][0].shape, xs[1][1].shape

(torch.Size([8, 3]),
 torch.Size([8, 2]),
 torch.Size([8, 1]),
 torch.Size([8, 1]))

In [ ]:
from tsai.data.validation import TimeSplitter
from tsai.data.core import TSRegression, get_ts_dls
X = np.repeat(np.repeat(np.arange(8)[:, None, None], 2, 1), 5, 2).astype(float)
X = np.concatenate([X, X])
y = np.concatenate([np.arange(len(X)//2)]*2)
alphabet = np.array(list(string.ascii_lowercase))
# y = alphabet[y]
splits = TimeSplitter(.5, show_plot=False)(range_of(X))
tfms = [None, TSRegression()]
dls1 = get_ts_dls(X, y, splits=splits, tfms=tfms)
dls1.one_batch()

(TSTensor(samples:8, vars:2, len:5, device=cuda:0),
 tensor([0., 1., 2., 3., 4., 5., 6., 7.], device='cuda:0'))

In [ ]:
data = np.concatenate([np.repeat(np.arange(8)[:, None], 3, 1)*np.array([1, 10, 100])]*2)
df = pd.DataFrame(data, columns=['cat1', 'cat2', 'cont'])
df['cont'] = df['cont'].astype(float)
df['target'] = y
cat_names = ['cat1', 'cat2']
cont_names = ['cont']
target = 'target'
dls2 = get_tabular_dls(df, procs=[Categorify, FillMissing, #Normalize
                                 ], cat_names=cat_names, cont_names=cont_names, y_names=target, splits=splits, bs=8)
dls2.one_batch()

(tensor([[1, 1],
         [2, 2],
         [5, 5],
         [4, 4],
         [6, 6],
         [3, 3],
         [8, 8],
         [7, 7]], device='cuda:0'),
 tensor([[  0.],
         [100.],
         [400.],
         [300.],
         [500.],
         [200.],
         [700.],
         [600.]], device='cuda:0'),
 tensor([[0],
         [1],
         [4],
         [3],
         [5],
         [2],
         [7],
         [6]], device='cuda:0', dtype=torch.int8))

In [ ]:
z = zip(_loaders[dls1.train.fake_l.num_workers == 0](dls1.train.fake_l))
for b in z: 
    print(b)
    break

((TSTensor(samples:8, vars:2, len:5, device=cpu), tensor([0., 1., 2., 3., 4., 5., 6., 7.])),)


In [ ]:
bs = 8
dls = get_mixed_dls(dls1, dls2, bs=bs)
dl = dls.train
xb, yb = dl.one_batch()
test_eq(len(xb), 2)
test_eq(len(xb[0]), bs)
test_eq(len(xb[1]), 2)
test_eq(len(xb[1][0]), bs)
test_eq(len(xb[1][1]), bs)
test_eq(xb[0].data[:, 0, 0].long(), xb[1][0][:, 0] - 1) # categorical data and ts are in synch
test_eq(xb[0].data[:, 0, 0], (xb[1][1]/100).flatten()) # continuous data and ts are in synch
test_eq(tensor(dl.input_idxs), yb.long().cpu())
dl = dls.valid
xb, yb = dl.one_batch()
test_eq(tensor(y[dl.input_idxs]), yb.long().cpu())

In [ ]:
#hide
out = create_scripts(); beep(out)

<IPython.core.display.Javascript object>

Converted 000_export.ipynb.
Converted 001_utils.ipynb.
Converted 010_data.validation.ipynb.
Converted 011_data.preparation.ipynb.
Converted 012_data.external.ipynb.
Converted 013_data.core.ipynb.
Converted 014_data.unwindowed.ipynb.
Converted 015_data.metadatasets.ipynb.
Converted 016_data.preprocessing.ipynb.
Converted 017_data.transforms.ipynb.
Converted 018_data.mixed_augmentation.ipynb.
Converted 019_data.image.ipynb.
Converted 020_data.features.ipynb.
Converted 021_data.tabular.ipynb.
Converted 022_data.mixed.ipynb.
Converted 050_losses.ipynb.
Converted 051_metrics.ipynb.
Converted 052_learner.ipynb.
Converted 052b_tslearner.ipynb.
Converted 053_optimizer.ipynb.
Converted 060_callback.core.ipynb.
Converted 061_callback.noisy_student.ipynb.
Converted 063_callback.MVP.ipynb.
Converted 064_callback.PredictionDynamics.ipynb.
Converted 100_models.layers.ipynb.
Converted 100b_models.utils.ipynb.
Converted 100c_models.explainability.ipynb.
Converted 101_models.ResNet.ipynb.
Converted 101